# Tensorflow

Primeiro precisamos importar as dependências e realizar os ajustes necessários nos dados a fim de deixá-los prontos para o processamento.

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train/255.0
x_test = x_test/255.0

Nos testes a seguir, adotamos um padrão de parametrização, no qual iremos variar cada parâmetro um a um a fim de entender seu impacto no modelo treinado, principalmente nas medidas de `loss` e `accuracy`.

Para fins de simplicidade, o código do treinamento será mostrado no primeiro teste de cada seção e ommitido nos demais, já que a alteração do parâmetro pode ser facilmente inferida.

A respeito dos parâmetros padrão temos: `epochs = 20`, `batch_size = 32`, `learning_rate = 0.01`, `optimizer = SGD`, `activation = None`, `hidden_layers = 1`, `neurons = 32`, `dropout = None`, `regularization = None`


## Configuração Padrão

In [ ]:
def get_metrics(x_test, y_test, model):
  y_pred = model.predict(x_test)
  y_pred = tf.argmax(y_pred,axis=1)
  classification = classification_report(y_test, y_pred, output_dict=True)
  return classification

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(32),
    Dense(32),
    Dense(10)
])

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/20
1500/1500 [==============================] - 10s 6ms/step - loss: 13.5195 - accuracy: 0.1008 - val_loss: 12.5035 - val_accuracy: 0.1035
Epoch 2/20
1500/1500 [==============================] - 4s 3ms/step - loss: 12.5287 - accuracy: 0.1019 - val_loss: 12.5035 - val_accuracy: 0.1035
Epoch 3/20
1500/1500 [==============================] - 7s 4ms/step - loss: 12.5287 - accuracy: 0.1019 - val_loss: 12.5035 - val_accuracy: 0.1035
Epoch 4/20
1500/1500 [==============================] - 6s 4ms/step - loss: 12.5287 - accuracy: 0.1019 - val_loss: 12.5035 - val_accuracy: 0.1035
Epoch 4: early stopping


In [ ]:
# Métricas para cada classe e métricas gerais já dentro do dicionário de resposta
print(get_metrics(x_test, y_test, model))

313/313 [==============================] - 1s 1ms/step
{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 980}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1135}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1032}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1010}, '4': {'precision': 0.0982, 'recall': 1.0, 'f1-score': 0.17883809870697503, 'support': 982}, '5': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 892}, '6': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 958}, '7': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1028}, '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 974}, '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1009}, 'accuracy': 0.0982, 'macro avg': {'precision': 0.009819999999999999, 'recall': 0.1, 'f1-score': 0.017883809870697504, 'support': 10000}, 'weighted avg': {'precision': 0.00964324, 'recall': 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Variação de batch size

De modo geral, podemos notar primeiramente o impacto do `batch_size` no tempo de treinamento do modelo. Quanto menor o `batch_size`, mais demorado é o treinamento pois cada unidade de processamento será responsável por um volume menor de dados, o que fará com que os pesos precisem ser ajustados com mais freqência. No entanto, apesar de mais lento, um `batch_size` menor tende a usar menos memória, já que as unidades podem realizar o processamento mais rápido.

Também foi possível notar que, embora os resultados alterando apenas o `batch_size` não tenham sido bons, foi com `batch_size=256` que o modelo obteve as melhores métricas. De um modo geral, setar `batch_size > 32` rendeu melhores resultados

### batch_size = 16

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(32),
    Dense(32),
    Dense(10)
])

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=16, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/20
3375/3375 [==============================] - 8s 2ms/step - loss: 3.7521 - accuracy: 0.1143 - val_loss: 3.7446 - val_accuracy: 0.1128
Epoch 2/20
3375/3375 [==============================] - 8s 3ms/step - loss: 3.8010 - accuracy: 0.1167 - val_loss: 3.7446 - val_accuracy: 0.1128
Epoch 3/20
3375/3375 [==============================] - 7s 2ms/step - loss: 3.8010 - accuracy: 0.1167 - val_loss: 3.7446 - val_accuracy: 0.1128
Epoch 4/20
3375/3375 [==============================] - 7s 2ms/step - loss: 3.8010 - accuracy: 0.1167 - val_loss: 3.7446 - val_accuracy: 0.1128
Epoch 4: early stopping


In [ ]:
# Métricas para cada classe e métricas gerais já dentro do dicionário de resposta
print(get_metrics(x_test, y_test, model))

313/313 [==============================] - 0s 1ms/step
{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 980}, '1': {'precision': 1.0, 'recall': 0.16651982378854627, 'f1-score': 0.2854984894259819, 'support': 1135}, '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1032}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1010}, '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 982}, '5': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 892}, '6': {'precision': 0.09764549994903679, 'recall': 1.0, 'f1-score': 0.17791809824496238, 'support': 958}, '7': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1028}, '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 974}, '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1009}, 'accuracy': 0.1147, 'macro avg': {'precision': 0.10976454999490368, 'recall': 0.11665198237885463, 'f1-score': 0.04634165876709443, 'support': 100

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### batch_size = 64

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(32),
    Dense(32),
    Dense(10)
])

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/20
844/844 [==============================] - 3s 3ms/step - loss: 5.2442 - accuracy: 0.1098 - val_loss: 5.3002 - val_accuracy: 0.0960
Epoch 2/20
844/844 [==============================] - 4s 4ms/step - loss: 5.2027 - accuracy: 0.1098 - val_loss: 5.3568 - val_accuracy: 0.1120
Epoch 3/20
844/844 [==============================] - 5s 6ms/step - loss: 5.2599 - accuracy: 0.1208 - val_loss: 5.3868 - val_accuracy: 0.1205
Epoch 4/20
844/844 [==============================] - 3s 3ms/step - loss: 5.2532 - accuracy: 0.1284 - val_loss: 5.3553 - val_accuracy: 0.1258
Epoch 4: early stopping


In [ ]:
# Métricas para cada classe e métricas gerais já dentro do dicionário de resposta
print(get_metrics(x_test, y_test, model))

313/313 [==============================] - 0s 1ms/step
{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 980}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1135}, '2': {'precision': 0.1032, 'recall': 1.0, 'f1-score': 0.18709209572153734, 'support': 1032}, '3': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1010}, '4': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 982}, '5': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 892}, '6': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 958}, '7': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1028}, '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 974}, '9': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1009}, 'accuracy': 0.1032, 'macro avg': {'precision': 0.01032, 'recall': 0.1, 'f1-score': 0.018709209572153735, 'support': 10000}, 'weighted avg': {'precision': 0.01065024, 'recall': 0.1032, 'f1-sc

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### batch_size = 256

In [ ]:
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(32),
    Dense(32),
    Dense(10)
])

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=256, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/20
211/211 [==============================] - 1s 4ms/step - loss: 1.9627 - accuracy: 0.4516 - val_loss: 2.5030 - val_accuracy: 0.6060
Epoch 2/20
211/211 [==============================] - 1s 3ms/step - loss: 2.2693 - accuracy: 0.4558 - val_loss: 2.3047 - val_accuracy: 0.4237
Epoch 3/20
211/211 [==============================] - 1s 4ms/step - loss: 2.3029 - accuracy: 0.3826 - val_loss: 2.3026 - val_accuracy: 0.4102
Epoch 4/20
211/211 [==============================] - 1s 5ms/step - loss: 2.3025 - accuracy: 0.3775 - val_loss: 2.3026 - val_accuracy: 0.4102
Epoch 5/20
211/211 [==============================] - 1s 5ms/step - loss: 2.3025 - accuracy: 0.3775 - val_loss: 2.3026 - val_accuracy: 0.4102
Epoch 6/20
211/211 [==============================] - 1s 6ms/step - loss: 2.3025 - accuracy: 0.3775 - val_loss: 2.3026 - val_accuracy: 0.4102
Epoch 6: early stopping


In [ ]:
# Métricas para cada classe e métricas gerais já dentro do dicionário de resposta
print(get_metrics(x_test, y_test, model))

313/313 [==============================] - 1s 1ms/step
{'0': {'precision': 1.0, 'recall': 0.003061224489795918, 'f1-score': 0.006103763987792472, 'support': 980}, '1': {'precision': 0.9805389221556886, 'recall': 0.5770925110132159, 'f1-score': 0.7265668330560178, 'support': 1135}, '2': {'precision': 0.8476454293628809, 'recall': 0.29651162790697677, 'f1-score': 0.4393395549174444, 'support': 1032}, '3': {'precision': 0.5864553314121037, 'recall': 0.402970297029703, 'f1-score': 0.47769953051643194, 'support': 1010}, '4': {'precision': 0.7927272727272727, 'recall': 0.4439918533604888, 'f1-score': 0.5691906005221933, 'support': 982}, '5': {'precision': 0.16443745082612116, 'recall': 0.9372197309417041, 'f1-score': 0.27978580990629187, 'support': 892}, '6': {'precision': 0.8089080459770115, 'recall': 0.5876826722338204, 'f1-score': 0.6807738814993953, 'support': 958}, '7': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1028}, '8': {'precision': 0.3425925925925926, 'recall': 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
